# Imports and connections 

In [1]:
import numpy as np
import pandas as pd

In [168]:
a, b = 1.5, 2.0
# a, b = 0.6, 1.1

i = 10
h = (b - a) / i

x_points = np.array([1.52, 1.97, 1.77])

def y(x: float) -> float:
    """
    Function to compute the given mathematical expression.

    Args:
    x (float): Input value

    Returns:
    float: Computed result
    """
    return np.pow(x, 2) + np.log(x) - 4

x_val = np.arange(a, a + h * (i + 1), h)
y_val = y(x_val)
x_val, y_val

(array([1.5 , 1.55, 1.6 , 1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  ]),
 array([-1.34453489, -1.15924507, -0.96999637, -0.77672471, -0.57937175,
        -0.37788421, -0.17221334,  0.03768564,  0.25185389,  0.47032937,
         0.69314718]))

### Сначала составим таблицу разделенных разностей для дальнейшей работы с ними

In [115]:
def count_finite_differences(y_values: np.ndarray) -> np.ndarray:
    """
    Calculates finite differences for grid function.

    Args:
    y_values (np.ndarray): y-values on the grid
    
    Returns:
    np.ndarray: Array of finite differences
    """
    n = len(y_values)
        
    finite_differences = [np.copy(y_values)]
        
    for j in range(1,n):
        current_row = np.zeros(n - j)
        for i in range(n - j):
            current_row[i] = finite_differences[j - 1][i + 1] - finite_differences[j - 1][i]
        finite_differences.append(current_row)
    return finite_differences

my_finite_differences = count_finite_differences(y_val)
print(pd.DataFrame(my_finite_differences).T)

          0         1         2         3         4             5   \
0  -1.344535  0.185290  0.003959  0.000064 -0.000006  6.658397e-07   
1  -1.159245  0.189249  0.004023  0.000058 -0.000005  5.720193e-07   
2  -0.969996  0.193272  0.004081  0.000053 -0.000005  4.936289e-07   
3  -0.776725  0.197353  0.004135  0.000049 -0.000004  4.277866e-07   
4  -0.579372  0.201488  0.004183  0.000045 -0.000004  3.722098e-07   
5  -0.377884  0.205671  0.004228  0.000041 -0.000003  3.250785e-07   
6  -0.172213  0.209899  0.004269  0.000038 -0.000003           NaN   
7   0.037686  0.214168  0.004307  0.000035       NaN           NaN   
8   0.251854  0.218475  0.004342       NaN       NaN           NaN   
9   0.470329  0.222818       NaN       NaN       NaN           NaN   
10  0.693147       NaN       NaN       NaN       NaN           NaN   

              6             7             8             9             10  
0  -9.382038e-08  1.542992e-08 -2.881783e-09  5.991776e-10 -1.366312e-10  
1  -7.839

**1-я формула Ньютона** по $x_0, x_1, x_2, \ldots, x_n$, $x = x_0 + th$, $0 < t < 1$,

$$
L_n(x_0 + th) = f_0 + t f_{1/2}^1 + \frac{t(t - 1)}{2} f_1^2 + \ldots + \frac{t(t - 1)\cdots(t - (n - 1))}{n!} f_{n/2}^n,
$$


In [116]:
def first_newton_interpolation(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Performs Newton interpolation.

    Args:
    x (float): The x-value to interpolate
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolated value
    """
    t = (x - x_values[0]) / (x_values[1] - x_values[0])
    
    if t < 0 or t > 1:
        raise ValueError("t must be close to start")
    
    finite_differences = count_finite_differences(y_values)
    n = len(y_values)
    
    res = finite_differences[0][0]
    for i in range(1, n):
        res += finite_differences[i][0] * np.prod([(t-k) / (k+1) for k in range(i)])  
        
    return res

In [118]:
res1 = first_newton_interpolation(x=x_points[0], x_values=x_val, y_values=y_val)
print(f"Результат: {res1}, \nИстинное значение: {y(x_points[0])},\nОшибка: {y(x_points[0]) - res1}") 

Результат: -1.2708896651422397, 
Истинное значение: -1.270889665141815,
Ошибка: 4.247713292215849e-13


**2-я формула Ньютона** по $x_0, x_1, x_2, \ldots, x_n$, $x = x_0 + th$, $-1 < t < 0$,

$$
L_n(x_0 + th) = f_0 + t f_{1/2}^1 + \frac{t(t + 1)}{2} f_1^2 + \ldots + \frac{t(t + 1)\cdots(t + (n - 1))}{n!} f_{n/2}^n,
$$

In [134]:
def second_newton_interpolation(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Performs Newton interpolation.

    Args:
    x (float): The x-value to interpolate
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolated value
    """
    t = (x - x_values[-1]) / (x_values[1] - x_values[0])
    
    if t > 0 or t < -1:
        raise ValueError("t must be close to end")
    
    finite_differences = count_finite_differences(y_values)
    n = len(y_values)
        
    res = finite_differences[0][-1]
    for i in range(1, n):
        res += finite_differences[i][-1] * np.prod([(t+k) / (k+1) for k in range(i)])  

    return res

In [120]:
res2 = second_newton_interpolation(x=x_points[1], x_values=x_val, y_values=y_val)
print(f"Результат: {res2}, \nИстинное значение: {y(x_points[1])},\nОшибка: {y(x_points[1]) - res2}")

Результат: 0.5589335427501104, 
Истинное значение: 0.5589335427498971,
Ошибка: -2.1327384303049257e-13


**1-я формула Гаусса** по точкам $x_0, x_1, x_{-1}, x_2, x_{-2}, \ldots, x_{n/2}, x_{-n/2}$,

$x = x_0 + th$, $0 < t \leq 0.5$,

$$
L_n(x_0 + th) = f_0 + t f_{1/2}^1 + \frac{t(t - 1)}{2} f_0^2 + \ldots + \frac{t(t^2 - 1)\cdots(t^2 - (\frac{n}{2} - 2)^2)(t^2 - (\frac{n}{2} - 1)^2)}{n!} f_0^n,
$$

In [172]:
def first_gauss_interpolation(x: float, x_values: np.ndarray, y_values: np.ndarray) -> float:
    """
    Performs Gauss interpolation.

    Args:
    x (float): The x-value to interpolate
    x_values (np.ndarray): Known x-values
    y_values (np.ndarray): Corresponding y-values
    
    Returns:
    float: Interpolated value
    """
    n = len(y_values)
    t = (x - x_values[n // 2]) / (x_values[1] - x_values[0])

    if not (0 < t < 0.5):
        raise ValueError("t must be close to middle")

    
    finite_differences = count_finite_differences(y_values)
    q = (x - x_values[n // 2]) / (x_values[1] - x_values[0])
    
        
    res = finite_differences[0][n // 2]
    for i in range(1, n):
        res += finite_differences[i][len(finite_differences[i] + 1) // 2] * np.prod([(q - k // 2) / (k + 1) if not k % 2 else (q + k // 2) / (k + 1) for k in range(i)])  

    return res

In [173]:
res3 = first_gauss_interpolation(x=x_points[2], x_values=x_val, y_values=y_val)
print(f"Результат: {res3}, \nИстинное значение: {y(x_points[2])},\nОшибка: {y(x_points[2]) - res3}")

Результат: -0.29528188701577507, 
Истинное значение: -0.29612045341426185,
Ошибка: -0.0008385663984867842


In [164]:
def omega(x):
    res = 1
    for x_i in x_val:
        res *= (x-x_i)
    return res

omega(x_points) * 41952.6 / 39916800, omega(x_points) * 1771.8 / 39916800 

(array([ 1.96767494e-12, -1.25016667e-12, -2.30304671e-14]),
 array([ 8.31015589e-14, -5.27987612e-14, -9.72654415e-16]))

### Оценка ошибки интерполяции 10-го порядка

Используем формулу остаточного члена интерполяционной формулы Лагранжа дессятого порядка:

$$R_2(x) = \frac{f^{XI}(\xi) \cdot \omega_{11}(x)}{11!}, \quad \xi \in [a,b]$$

где  

$$
\omega_{11}(x) = (x - x_0) ... (x - x_{10})
$$

#### 1. Вычислим третью производную функции

Функция:

$$
f(x) = x^2 + \ln(x) - 4
$$
11-я производная:

$$
f^{XI}(x) = \frac{10!}{x^11}
$$

$$
f^{XI}(1.5) = f{X|}_{\max} \approx 41952.6
$$

$$
f^{XI}(2) = f{X|}_{\min} = 1771.8
$$

$$
\omega_{XI}(1.52) = 1.87219117e-09
$$

$$
\omega_{XI}(1.97) = -1.18950084e-09
$$

$$
\omega_{XI}(1.77) = -2.19128862e-11
$$

$$
|R_{X}|_{\max}(1.52) = \left| \frac{f^{XI}_{\max} \cdot \omega_{XI}(1.52)}{11!} \right| = \left| \frac{41952.6 \cdot 1.87219117e-09}{11!} \right| \approx 1.96767494e-12
$$

$$
|R_{X}|_{\min}(1.52) = \left| \frac{f^{XI}_{\min} \cdot \omega_{XI}(1.52)}{11!} \right| = \left| \frac{1771.8 \cdot 1.87219117e-09}{11!} \right| \approx 8.31015589e-14
$$

$$
|R_{X}|_{\max}(1.97) = \left| \frac{f^{XI}_{\max} \cdot \omega_{XI}(1.97)}{11!} \right| \approx -1.25016667e-12
$$

$$
|R_{X}|_{\min}(1.97) = \left| \frac{f^{XI}_{\min} \cdot \omega_{XI}(1.97)}{11!} \right|  \approx -5.27987612e-14
$$

$$
|R_{X}|_{\max}(1.77) = \left| \frac{f^{XI}_{\max} \cdot \omega_{XI}(1.77)}{11!} \right| \approx -2.30304671e-14
$$

$$
|R_{X}|_{\min}(1.77) = \left| \frac{f^{XI}_{\min} \cdot \omega_{XI}(1.77)}{11!} \right|  \approx -9.72654415e-16
$$


### Все вычисленные ошибки попадают в указанные интервалы, кроме вычислений при помощи третьей формулы